In [2]:
import import_ipynb
import math
from torch import nn
from torch import optim
from tqdm import tqdm

from preprocessing import *


importing Jupyter notebook from preprocessing.ipynb


In [3]:
if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

In [4]:
def train(model, data, optimizer, loss_function, train_size=0.8, n_epochs=1, lookback_window=50):
    
    a, b, c = data.size()
    
    train_size = int(train_size * a) - lookback_window
    test_size = a - train_size - lookback_window
    
    for epoch in range(1, n_epochs+1):
        
        train_loss = 0
        test_loss = 0
        
        for i in tqdm(range(a - lookback_window)):
    
            optimizer.zero_grad()
        
            X = data[i : i+lookback_window]
            y = data[i+lookback_window]

            out = model(X)
            loss = loss_function(out, y)
                
            if i < train_size:
                
                train_loss += loss
                loss.backward()
                optimizer.step()
                
            else:
                
                test_loss += loss
            
        print("Epoch {} completed. \nTraining Loss: {:.3f}".format(epoch, train_loss / (train_size)))
        print("Test Loss: {:.3f}".format(test_loss / (test_size)))        

In [11]:
class LSTM(nn.Module):
    
    def __init__(self, in_parameters, hidden_parameters, linear_parameters, out_parameters, num_layers=1, dropout=0.5, batch_size=1, activation=False):
        
        super(LSTM, self).__init__()
        
        self.rnn = nn.LSTM(in_parameters, hidden_parameters, num_layers)
        self.linear1 = nn.Linear(hidden_parameters, linear_parameters)
        self.linear2 = nn.Linear(linear_parameters, out_parameters)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        
        self.hidden_parameters = hidden_parameters
        self.out_parameters = out_parameters
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.activation = activation
    
    def init_hidden(self):
        
        self.hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_parameters)
        self.cell = torch.zeros(self.num_layers, self.batch_size, self.hidden_parameters)
        
        if torch.cuda.is_available():
            self.hidden = self.hidden.cuda()
            self.cell = self.cell.cuda()
            
    def forward(self, X):
        
        self.init_hidden()
        x = torch.tensor((X.size()))
        for i in range(X.size()[0]):
            x, (self.hidden, self.cell) = self.rnn(X[i:i+1], (self.hidden, self.cell)) 
        X = x
        X = X.view((1, -1))
        X = self.linear1(X)
        X = self.tanh(X)
        X = self.linear2(X)
        
        
        if self.activation:
            X = self.activation(X)
        
        return X
    
# SAMPLE USAGE

# model = LSTM(9, 20, 20, 9)
# model = model.cuda()
# X = torch.randn((50, 1, 9))
# X = X.cuda()
# print(model(X))
        

tensor([[[-0.0170, -0.0040,  0.2274,  0.2695, -0.0939,  0.2418,  0.2621,
          -0.1861,  0.0557]]], device='cuda:0', grad_fn=<AddBackward0>)


In [10]:
class CNN(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, in_parameters, activation=None):
        
        super(CNN, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.in_parameters = in_parameters
        self.conv_out_size = out_channels * (in_parameters - kernel_size + 1)**2
        self.activation = activation
        self.tanh = nn.Tanh()
        
        self.linear1 = nn.Linear(in_parameters, in_parameters*in_parameters)
        self.cnn = nn.Conv2d(in_channels, out_channels, kernel_size)
        self.linear2 = nn.Linear(self.conv_out_size, in_parameters*in_parameters)
        self.linear3 = nn.Linear(in_parameters*in_parameters, in_parameters)
        
        
    def forward(self, X):
        
        X = self.linear1(X)
        X = X.view((1, self.in_channels, self.in_parameters, self.in_parameters))
        X = self.tanh(X)
        X = self.cnn(X)
        X = X.view((1, -1))
        X = self.linear2(X)
        X = self.tanh(X)
        X = self.linear3(X)
        
        if self.activation:
            X = self.activation(X)
            
        
        return X

model = CNN(10, 16, 3, 9)

X = torch.randn((10, 1, 9))
print(X.size())

X = model(X)
print(X.size())
        
    
    

torch.Size([10, 1, 9])
torch.Size([1, 9])


In [11]:
class CNN_LSTM(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, in_parameters, hidden_parameters, num_layers, batch_size=1, activation=None):
        
        super(CNN_LSTM, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.in_parameters = in_parameters
        self.rnn_in = (in_parameters - kernel_size + 1)**2
        self.conv_out_size = out_channels * (in_parameters - kernel_size + 1)**2
        self.activation = activation
        self.hidden_parameters = hidden_parameters
        self.num_layers = num_layers
        self.batch_size = batch_size
        
        self.linear1 = nn.Linear(in_parameters, in_parameters*in_parameters)
        self.cnn = nn.Conv2d(in_channels, out_channels, kernel_size)
        self.linear2 = nn.Linear(self.conv_out_size, in_parameters*in_parameters)
        self.linear3 = nn.Linear(hidden_parameters, in_parameters)
        self.rnn = nn.LSTM(self.rnn_in, hidden_parameters, num_layers)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
       
        
    def init_hidden(self):
        
        self.hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_parameters)
        self.cell = torch.zeros(self.num_layers, self.batch_size, self.hidden_parameters)

        if torch.cuda.is_available():
            self.hidden = self.hidden.cuda()
            self.cell = self.cell.cuda()
                
    def forward(self, X):
        
        X = self.linear1(X)
        X = X.view((1, self.in_channels, self.in_parameters, self.in_parameters))
        X = self.cnn(X)
        X = X[0]
        X = X.view((X.size()[0], 1, -1))
        X = self.relu(X)
        X, _ = self.rnn(X)
        X = X[-1]
        X = self.relu(X)
        X = self.linear3(X)
        
        if self.activation:
            
            X = self.activation(X)
            
        return X


model = CNN_LSTM(in_channels=10, out_channels=16, kernel_size=3, in_parameters=9, hidden_parameters=50, num_layers=2)

X = torch.randn((10, 1, 9))
print(X.size())

X = model(X)
print(X.size())

torch.Size([10, 1, 9])
torch.Size([1, 9])


In [12]:
class ANN(nn.Module):
    
    def __init__(self, in_params, hidden1, hidden2, out_params):
        
        super(ANN, self).__init__()
        
        self.linear1 = nn.Linear(in_params, hidden1)
        self.linear2 = nn.Linear(hidden1, hidden2)
        self.linear3 = nn.Linear(hidden2, out_params)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, X):
        
        X = X.view(-1)
        X = self.linear1(X)
        X = self.relu(X)
        X = self.linear2(X)
        X = self.relu(X)        
        X = self.linear3(X)
        X = self.tanh(X)
        X = X.view((1 ,-1))
        
        return X
    
model = ANN(90, 100, 200, 90)

X = torch.randn((10, 1, 9))
print(X.size())

X = model(X)
print(X.size())

torch.Size([10, 1, 9])
torch.Size([1, 90])
